In [1]:
import warnings
warnings.filterwarnings('ignore')

from langchain_community.llms import Ollama
from crewai import Agent, Crew, Task
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Initialize the LLM
llm = Ollama(
    model="llama3",
    base_url="http://localhost:11434"
)

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "With a keen sense of space and understanding of event logistics, "
        "you excel at finding and securing the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

# Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal="Manage all logistics for the event including catering and equipment",
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "Organized and detail-oriented, you ensure that every logistical aspect of the event "
        "from catering to equipment setup is flawlessly executed to create a seamless experience."
    )
)

# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "Creative and communicative, you craft compelling messages and "
        "engage with potential attendees to maximize event exposure and participation."
    )
)
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}
result = event_management_crew.kickoff(inputs=event_details)
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)
from IPython.display import Markdown
Markdown("marketing_report.md")

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
Thought: I need to start by gathering more information about the Tech Innovation Conference. What are the key criteria for this event?

Action: Search the internet
Action Input: {"query": "San Francisco venues for tech conferences", "context": "Looking for a venue that can accommodate 500 attendeesI apologize for the mistake. Let me retry.

Thought: 
Action: Search the internet
Action Input: {"search_query": "San Francisco venues for tech conferences", "context": "Looking for a venue that can accommodate 500 attendeesI apologize for the mistake! Let me retry with the correct input.

Thought: 
Action: Search the internet
Action Input: {"search_query": "San Francisco venues for tech conferences", "context": "Looking for a venue that can accommodate 500 attendeesI apologize for the mistake. Le

marketing_report.md

Thought: To effectively market the Tech Innovation Conference and engage at least 500 potential attendees, I need to gather information about the event and its target audience.

Action: Search the internet
Action Input: {"search_query": "Tech Innovation ConferenceThought: I need to gather information about catering options for an event with 500 participants.

Action: Search the internet
Action Input: {"search_query": "catering options for events with 500 participantsI apologize for the mistake. Let me try again!

Thought: To effectively market the Tech Innovation Conference and engage at least 500 potential attendees, I need to gather information about the event and its target audience.

Action: Read website content
Action Input: {"website_url": "https://techinnovationconference.comI apologize for the mistake. Let me try again!

Thought: 
Action: Search the internet
Action Input: {"search_query": "catering options for events with 500 participantsI apologize for the mistake earlier. Let